# 麥克連大盤脈動_part_1_大盤指數收盤資訊   
* 大盤指數收盤資訊資料來源 [每日市場成交資訊](https://www.twse.com.tw/zh/trading/historical/fmtqik.html)  

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import duckdb

In [2]:
from finlab import data

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.finlab_api import finlab_manager as flm
from proj_util_pkg.common.duckdb_tool import insert_dataframe_to_duckdb

## 公用參數設定

In [4]:
pd.set_option("display.max_columns", None)

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

In [6]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [7]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

## 大盤指數收盤資訊  
* 大盤指數收盤資訊資料來源 [每日市場成交資訊](https://www.twse.com.tw/zh/trading/historical/fmtqik.html)  

In [8]:
# 讀取台股加權指數
market_info_index = data.get('market_transaction_info:收盤指數', save_to_storage=True)
market_info_index.rename(columns={'OTC': 'OTC指數', 'TAIEX': 'TAIEX指數'}, inplace=True)
market_info_index.reset_index(inplace=True)

In [9]:
# 計算TAIEX漲跌數
market_info_index['TAIEX_漲跌數'] = market_info_index['TAIEX指數'].diff()

# 依據TAIEX指數漲跌幅度
market_info_index['TAIEX_漲跌幅'] = market_info_index['TAIEX指數'].diff() / market_info_index['TAIEX指數'].shift(1) * 100

# 計算OTC漲跌數
market_info_index['OTC_漲跌數'] = market_info_index['OTC指數'].diff()

# 計算OTC指數漲跌幅度
market_info_index['OTC_漲跌幅'] = market_info_index['OTC指數'].diff() / market_info_index['OTC指數'].shift(1) * 100


In [ ]:
market_info_index

In [11]:
# 讀取台股成交金額資訊
market_info_amount = data.get('market_transaction_info:成交金額', save_to_storage=True)
market_info_amount.rename(columns={'OTC': 'OTC成交金額', 'TAIEX': 'TAIEX成交金額'}, inplace=True)

# market_info_amount金額換算成億元
market_info_amount['OTC成交金額'] = market_info_amount['OTC成交金額'] / 100000000
market_info_amount['TAIEX成交金額'] = market_info_amount['TAIEX成交金額'] / 100000000

market_info_amount.reset_index(inplace=True)


In [ ]:
market_info_amount

In [ ]:
# 使用 pandas 合併資料，然後存入 DuckDB
# 依據 date 欄位 join market_info_index 與 market_info_amount
merged_data = pd.merge(market_info_index, market_info_amount, on='date', how='inner')

# 依據日期反向排序，取出最近30筆紀錄
merged_data_sorted = merged_data.sort_values('date', ascending=False).head(30)

# 選擇需要的欄位並重新命名
tw_stock_barometer_part_01 = merged_data_sorted[[
    'date', 
    'TAIEX指數', 'TAIEX_漲跌數', 'TAIEX_漲跌幅', 'TAIEX成交金額', 
    'OTC指數', 'OTC_漲跌數', 'OTC_漲跌幅', 'OTC成交金額'
]].copy()

# 重新命名 date 欄位為 Date
tw_stock_barometer_part_01.rename(columns={'date': 'Date'}, inplace=True)

# 使用工具函數將資料存入 DuckDB
table_name = 'tw_sotck_barometer_part_01'
insert_dataframe_to_duckdb(
    df=tw_stock_barometer_part_01,
    table_name=table_name,
    conn=conn_duckdb,
    if_exists='replace'  # 如果表已存在則替換
)

print(f"✅ 成功將最近 {len(tw_stock_barometer_part_01)} 筆資料存入表 {table_name}")
print(f"資料時間範圍: {tw_stock_barometer_part_01['Date'].min()} 至 {tw_stock_barometer_part_01['Date'].max()}")
print(f"原始資料總筆數: {len(merged_data)} 筆，篩選後: {len(tw_stock_barometer_part_01)} 筆")

In [ ]:
# 查詢表中所有資料
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc LIMIT 10").fetch_df()

In [15]:
# 關閉資料庫連線
conn_duckdb.close()
